In [2]:
library(tidyverse)

# Read a fixed width file into a tibble

A fixed width file can be a very compact representation of numeric data. It's also very fast to parse, because every field is in the same place in every line. Unfortunately, it's painful to parse because you need to describe the length of every field. Readr aims to make it as easy as possible by providing a number of different ways to describe the field structure.

```r
read_fwf(
  file,
  col_positions,
  col_types = NULL,
  locale = default_locale(),
  na = c("", "NA"),
  comment = "",
  trim_ws = TRUE,
  skip = 0,
  n_max = Inf,
  guess_max = min(n_max, 1000),
  progress = show_progress(),
  skip_empty_rows = TRUE
)

fwf_empty(
  file,
  skip = 0,
  skip_empty_rows = FALSE,
  col_names = NULL,
  comment = "",
  n = 100L
)

fwf_widths(widths, col_names = NULL)

fwf_positions(start, end, col_names = NULL)

fwf_cols(...)
```

# Examples

In [3]:
fwf_sample <- readr_example("fwf-sample.txt")
writeLines(read_lines(fwf_sample))

John Smith          WA        418-Y11-4111
Mary Hartford       CA        319-Z19-4341
Evan Nolan          IL        219-532-c301


You can specify column positions in several ways:
* 1. Guess based on position of empty columns

In [7]:
position <- fwf_empty(fwf_sample, col_names = c('First name', 'Last name', 'State', 'snn'))
position

$begin
[1]  0  5 20 30

$end
[1]  4 13 22 NA

$skip
[1] 0

$col_names
[1] "First name" "Last name"  "State"      "snn"

In [8]:
read_fwf(fwf_sample, position)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  `First name` = col_character(),
  `Last name` = col_character(),
  State = col_character(),
  snn = col_character()
)



First name,Last name,State,snn
John,Smith,WA,418-Y11-4111
Mary,Hartford,CA,319-Z19-4341
Evan,Nolan,IL,219-532-c301


* 2. A vector of field widths

In [14]:
position <- fwf_widths(c(20, 10, 12), col_names = c('name', 'state', 'snn'))
position

begin,end,col_names
0,20,name
20,30,state
30,42,snn


In [16]:
read_fwf(fwf_sample, position)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  name = col_character(),
  state = col_character(),
  snn = col_character()
)



name,state,snn
John Smith,WA,418-Y11-4111
Mary Hartford,CA,319-Z19-4341
Evan Nolan,IL,219-532-c301


* 3. Paired vectors of start and end positions

In [18]:
position <- fwf_positions(start = c(1, 30), end = c(20, 42), col_names = c('name', 'snn'))
position

begin,end,col_names
0,20,name
29,42,snn


In [19]:
read_fwf(fwf_sample, position)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  name = col_character(),
  snn = col_character()
)



name,snn
John Smith,418-Y11-4111
Mary Hartford,319-Z19-4341
Evan Nolan,219-532-c301


* 4. Named arguments with start and end positions

In [21]:
position <- fwf_cols(name = c(1, 20), snn = c(30, 42))
position

begin,end,col_names
0,20,name
29,42,snn


In [22]:
read_fwf(fwf_sample, position)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  name = col_character(),
  snn = col_character()
)



name,snn
John Smith,418-Y11-4111
Mary Hartford,319-Z19-4341
Evan Nolan,219-532-c301


* 5. Named arguments with column widths

In [23]:
position <- fwf_cols(name = 20, state = 10, snn = 12)
position

begin,end,col_names
0,20,name
20,30,state
30,42,snn


In [24]:
read_fwf(fwf_sample, position)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  name = col_character(),
  state = col_character(),
  snn = col_character()
)



name,state,snn
John Smith,WA,418-Y11-4111
Mary Hartford,CA,319-Z19-4341
Evan Nolan,IL,219-532-c301


# Arguments

### `col_positions`

Column positions, as created by `fwf_empty()`, `fwf_widths()` or `fwf_positions()`. To read in only selected fields, use `fwf_positions()`. If the width of the last column is variable (a ragged fwf file), supply the last end position as NA.

<hr>

In [27]:
TF <- 'VN Pikachu       31       VN Champions
ETOGRUL          34       Dirilis
Monkey           35       Daemones'

In [28]:
cat(TF)

VN Pikachu       31       VN Champions
ETOGRUL          34       Dirilis
Monkey           35       Daemones

* Based on empty columns

In [36]:
position <- fwf_empty(TF, col_names = c('Name', 'Level', 'Clan'))
position

$begin
[1]  0 17 26

$end
[1] 10 19 NA

$skip
[1] 0

$col_names
[1] "Name"  "Level" "Clan"

In [37]:
read_fwf(TF, position)

Name,Level,Clan
VN Pikachu,31,VN Champions
ETOGRUL,34,Dirilis
Monkey,35,Daemones


*  Based on column widths of each field

In [47]:
position <- fwf_widths(c(17, 2, NA), col_names = c('Name', 'Level', 'Clan'))

position

begin,end,col_names
0,17,Name
17,19,Level
19,NA,Clan


In [48]:
read_fwf(TF, position)

Name,Level,Clan
VN Pikachu,31,VN Champions
ETOGRUL,34,Dirilis
Monkey,35,Daemones


In [49]:
# equivalent
position <- fwf_cols(Name = 17, Level = 2, Clan = NA)

read_fwf(TF, position)

Name,Level,Clan
VN Pikachu,31,VN Champions
ETOGRUL,34,Dirilis
Monkey,35,Daemones


* Based on the start and end positon of each column

In [52]:
position <- fwf_positions(start = c(1, 20), end = c(17, NA), col_names = c('Name', 'Clan'))
position

begin,end,col_names
0,17,Name
19,NA,Clan


In [53]:
read_fwf(TF, position)

Name,Clan
VN Pikachu,VN Champions
ETOGRUL,Dirilis
Monkey,Daemones


In [54]:
# equivalent
position <- fwf_cols(Name = c(1,17), Clan = c(20, NA))

read_fwf(TF, position)

Name,Clan
VN Pikachu,VN Champions
ETOGRUL,Dirilis
Monkey,Daemones


### `n`

Number of lines the tokenizer will read to determine file structure. By default it is set to 100.

### `widths`	

Width of each field. Use NA as width of last field when reading a ragged fwf file.

### `start`, `end`	

Starting and ending (inclusive) positions of each field. Use NA as last end field when reading a ragged fwf file.

### `...`	

If the first element is a data frame, then it must have all numeric columns and either one or two rows. The column names are the variable names. The column values are the variable widths if a length one vector, and if length two, variable start and end positions. The elements of ... are used to construct a data frame with or or two rows as above.

### other arguments

<b><a href = './delimited file.ipynb'>See this notebook</a></b>